## Question 1
- Use Transformers model and build a Code Generation application on [evolved codealpaca](https://huggingface.co/datasets/theblackcat102/evol-codealpaca-v1) dataset

#### Fine-Tuning LLMs using Qlora

In this notebook, we fine-tuned [Orca-3b](https://huggingface.co/pankajmathur/orca_mini_3b) on evolved codealpaca to perform Code Generation tasks. This is an example on how we can leverage on Colab's free T4 GPU to fine-tune LLMs using QloRA.

#### Setup

Run the cells below to setup and install the required libraries. For our experiment we will need `accelerate`, `peft`, `transformers`, `datasets`, `scipy` and `TRL` to leverage `SFTTrainer`. We will use `bitsandbytes` to quantize the base model into 4bit. We will also install `einops` but it is mainly used for loading falcon.

In [1]:
!pip install transformers==4.30
!pip install -q -U trl accelerate sentencepiece git+https://github.com/huggingface/peft.git
!pip install -q -U datasets bitsandbytes einops scipy wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 97.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 99.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.0/100.0 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/

### Loading the Evolved codealpaca dataset from HuggingFace

In [ ]:
from datasets import load_dataset

dataset_name = 'theblackcat102/evol-codealpaca-v1'
dataset = load_dataset(dataset_name, split="train")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

Dataset({
    features: ['instruction', 'output'],
    num_rows: 111272
})

In [ ]:
dataset[0]

{'instruction': "Please amend the subsequent Python script so that it includes a 'while' loop rather than the existing 'for' loop, which iterates through the items of an integer list.\n\nThe script currently has a bug where it attempts to print an object that is outside the bounds of the list. Fix this error and modify the script to use 'while' instead of 'for' loop. Ensure your script correctly handles empty lists. \n\n```python\n  # Establish an integer list\n  arr = [1, 2, 3, 4]\n\n  # Determine the length of the list\n  n = len(arr)\n\n  # Traverse the list and output each individual element\n  for i in range(n+1):\n      print(arr[i])\n```",
 'output': '```python\n# Establish an integer list\narr = [1, 2, 3, 4]\n\n# Determine the length of the list\nn = len(arr)\n\n# Initialize index at 0\ni = 0\n\n# Traverse the list and output each individual element\nwhile i < n:\n    print(arr[i])\n    i += 1\n```\nIn the given code, it tries to access `arr[n]` which is out of bounds as python

In [ ]:
dataset['instruction'][0]

"Please amend the subsequent Python script so that it includes a 'while' loop rather than the existing 'for' loop, which iterates through the items of an integer list.\n\nThe script currently has a bug where it attempts to print an object that is outside the bounds of the list. Fix this error and modify the script to use 'while' instead of 'for' loop. Ensure your script correctly handles empty lists. \n\n```python\n  # Establish an integer list\n  arr = [1, 2, 3, 4]\n\n  # Determine the length of the list\n  n = len(arr)\n\n  # Traverse the list and output each individual element\n  for i in range(n+1):\n      print(arr[i])\n```"

We will subset the dataset to only 1000 samples due to the duration limitations on Colab's free usage of their T4 GPU

In [ ]:
# subset dataset to 1000 samples due to the limitation on Colab's free tier GPU usage
samp_dataset = dataset.shuffle(seed=42).select(range(1000))

len(samp_dataset)

1000

### Loading the model

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer
from accelerate import Accelerator

model_name = "psmathur/orca_mini_3b"
# model_name = "microsoft/phi-1_5"

bnb_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
  model_name,
  quantization_config=bnb_config,
  trust_remote_code=True
)
model.config.use_cache = False

(…)ur/orca_mini_3b/resolve/main/config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

(…)esolve/main/pytorch_model.bin.index.json:   0%|          | 0.00/21.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/3.72G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

(…)i_3b/resolve/main/generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

Let's also load the tokenizer

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

(…)ni_3b/resolve/main/tokenizer_config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/534k [00:00<?, ?B/s]

(…)_3b/resolve/main/special_tokens_map.json:   0%|          | 0.00/208 [00:00<?, ?B/s]

Asking orca to generate scikit-learn linear regression codes before fine-tuning

In [7]:
%%time

#generate text function
def generate_text(system, instruction, input=None):

    if input:
        prompt = f"### System:\n{system}\n\n### User:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n"
    else:
        prompt = f"### System:\n{system}\n\n### User:\n{instruction}\n\n### Response:\n"

    tokens = tokenizer.encode(prompt)
    tokens = torch.LongTensor(tokens).unsqueeze(0)
    tokens = tokens.to('cuda')

    instance = {'input_ids': tokens,'top_p': 1.0, 'temperature':0.7, 'generate_len': 1024, 'top_k': 50}

    length = len(tokens[0])
    with torch.no_grad():
        rest = model.generate(
            input_ids=tokens,
            max_length=length+instance['generate_len'],
            use_cache=True,
            do_sample=True,
            top_p=instance['top_p'],
            temperature=instance['temperature'],
            top_k=instance['top_k']
        )
    output = rest[0][length:]
    string = tokenizer.decode(output, skip_special_tokens=True)
    return f'[!] Response: {string}'

# Sample test instruction used by Youtuber Sam Witteveen https://www.youtube.com/@samwitteveenai
system = 'You are an AI assistant that follows instruction extremely well. Help as much as you can.'
instruction = 'Can you help me with scikit-learn linear regression codes'
print(generate_text(system, instruction))


[!] Response: Yes, I can help you with scikit-learn linear regression codes. However, I will need some more information about your problem to provide you with the correct code. Can you please provide me with the following details:

1. What is the input data that you want to use for the linear regression?
2. What is the target variable that you want to predict?
3. What is the number of features that you want to use for the linear regression?
4. What is the type of data that you want to use for the linear regression (numeric, categorical, etc.)?
5. What is the output that you want scikit-learn to give you after running the linear regression code?

Once you provide me with the above information, I can help you write the scikit-learn linear regression code for your problem.
CPU times: user 14.5 s, sys: 29.2 ms, total: 14.5 s
Wall time: 15.1 s


In [ ]:
from peft import LoraConfig, get_peft_model

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
  lora_alpha=lora_alpha,
  lora_dropout=lora_dropout,
  r=lora_r,
  bias="none",
  task_type="CAUSAL_LM"
)

### Loading the trainer
We will use the [SFTTrainer](https://huggingface.co/docs/trl/sft_trainer) from `TRL` library that gives a wrapper around transformers Trainer to easily fine-tune models on instruction based datasets using `PEFT` adapters. Let's first load the training arguments below.

In [ ]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 1
gradient_accumulation_steps = 1
optim = "paged_adamw_32bit"
save_steps = 100
logging_steps = 10
learning_rate = 4e-3
max_grad_norm = 0.3
max_steps = -1
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
  output_dir=output_dir,
  per_device_train_batch_size=per_device_train_batch_size,
  gradient_accumulation_steps=gradient_accumulation_steps,
  optim=optim,
  save_steps=save_steps,
  logging_steps=logging_steps,
  learning_rate=learning_rate,
  fp16=True,
  max_grad_norm=max_grad_norm,
  max_steps=max_steps,
  warmup_ratio=warmup_ratio,
  group_by_length=True,
  lr_scheduler_type=lr_scheduler_type,
  num_train_epochs=1,
)

Then finally pass everything to the trainer

In [ ]:
from trl import SFTTrainer

def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['instruction'])):
        text = f"### Instruction: {example['instruction'][i]}\n ### Response: {example['output'][i]}"
        output_texts.append(text)
    return output_texts

# def formatting_prompts_func(examples):
#     output_text = []
#     for i in range(len(examples["instruction"])):
#         instruction = examples["instruction"][i]
#         response = examples["output"][i]

#         text = f'''Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

#         ### Instruction:
#         {instruction}

#         ### Response:
#         {response}
#         '''

#         output_text.append(text)
#     return output_text

max_seq_length = 2048

trainer = SFTTrainer(
  model=model,
  train_dataset=samp_dataset,
  peft_config=peft_config,
  formatting_func=formatting_prompts_func, # instruction dataset usually contains more than one column (instruction + output)
  #dataset_text_field="text",  # for non-instruction dataset
  max_seq_length=max_seq_length,
  tokenizer=tokenizer,
  args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:107: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

We will also pre-process the model by upcasting the layer norms in float 32 for more stable training

In [ ]:
for name, module in trainer.model.named_modules():
  if "norm" in name:
    module = module.to(torch.float32)

### Train the model
Now let's train the model! Simply call `trainer.train()`

In [ ]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,1.878100
20,1.884600
30,1.839900
40,1.582700
50,1.763000
60,2.468700
70,1.830700
80,1.414700
90,1.566600
100,1.427500


TrainOutput(global_step=1000, training_loss=1.910161961555481, metrics={'train_runtime': 2321.9024, 'train_samples_per_second': 0.431, 'train_steps_per_second': 0.431, 'total_flos': 7181226034752000.0, 'train_loss': 1.910161961555481, 'epoch': 1.0})

The `SFTTrainer` will take care of properly saving only the adapters during training instead of saving the entire model

In [ ]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model # Take care of distributed/parallel training
model_to_save.save_pretrained("outputs")

In [ ]:
lora_config = LoraConfig.from_pretrained('outputs')
model = get_peft_model(model, lora_config)

Prompting the fine-tuned model to perform code generation

In [ ]:
%%time

text = '''### User: I need codes for cars detection using deep learning\n
### Assistant: '''
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt", return_token_type_ids=False).to(device)
outputs = model.generate(**inputs, max_new_tokens=1024)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


### User: I need codes for cars detection using deep learning

### Assistant: 
Here are some codes for cars detection using deep learning:

1. MobileNet: 

```
import tensorflow as tf
from tensorflow_keras.models import Sequential
from tensorflow_keras.layers import Dense, Flatten, Conv2D, MaxPooling2D

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=10, batch_size=32)
```

2. VGG16:

```
import tensorflow as tf
from tensorflow_keras.models import Sequential
from tensorflow_keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential()
model.add(Conv2D(64, (3, 3), act

In [ ]:
%%time

text = '''### User: Can you help me with scikit-learn linear regression codes?\n
### Assistant: '''
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt", return_token_type_ids=False).to(device)
outputs = model.generate(**inputs, max_new_tokens=1024)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


### User: Can you help me with scikit-learn linear regression codes?

### Assistant: 
Sure, I can help you with scikit-learn linear regression codes. Here's an example code for a simple linear regression model:

```python
import pandas as pd
from sklearn.linear_model import LinearRegression

# Load the data
data = pd.read_csv('data.csv')

# Split the data into features and target
X = data.drop('target', axis=1)
y = data['target']

# Create a linear regression object
model = LinearRegression()

# Fit the model to the data
model.fit(X, y)

# Print the coefficients of the model
print(model.coef_)
```

This code loads a dataset from a CSV file, splits it into features and target, creates a linear regression object, fits the model to the data, and then prints the coefficients of the model. You can modify this code to work with your own data and customize the model to fit your specific needs.
CPU times: user 1min 58s, sys: 244 ms, total: 1min 58s
Wall time: 2min 4s



---



## Question 2

- Use LangChain to build an ChatGPT-like ChatBot application


In [ ]:
!pip install langchain chromadb pypdf sentence_transformers InstructorEmbedding streamlit bitsandbytes ctransformers[cuda] accelerate einops safetensors xformers
!pip install -q -U git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 448.1/448.1 kB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.6/276.6 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 110.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 114.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 MB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 4.1 MB/s e

*** Note:

`Restart runtime` if Colab complains about missing packages that was `pip install` above

### Import

In [ ]:
import torch
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain.llms import LlamaCpp

from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

### Define LLM

In [ ]:
from langchain.llms import CTransformers

# from ctransformers import AutoConfig
# from ctransformers import AutoModelForCausalLM

# config = AutoConfig.from_pretrained("TheBloke/Mistral-7B-v0.1-GGUF")
# config.config.max_new_tokens = 2000
# config.config.context_length = 4000

# llm = AutoModelForCausalLM.from_pretrained("TheBloke/Mistral-7B-v0.1-GGUF", model_file="mistral-7b-v0.1.Q5_K_M.gguf", model_type="mistral",gpu_layers=0, config=config)

config = {'max_new_tokens': 1024, 'temperature': 0, 'context_length': 1024}
llm = CTransformers(model='TheBloke/Mistral-7B-Instruct-v0.1-GGUF',model_file="mistral-7b-instruct-v0.1.Q4_K_M.gguf", config=config, n_ctx=4096)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

### Loading PDF documents

In [ ]:
# Load and process the text files
# loader = TextLoader('single_text_file.txt')
loader = DirectoryLoader('/content/drive/MyDrive/colab/AI Planet LLM Bootcamp/data', glob="./*.pdf", loader_cls=PyPDFLoader)

documents = loader.load()

In [ ]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=300)
texts = text_splitter.split_documents(documents)

In [ ]:
len(texts)

13

In [ ]:
instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-large",
                                                      model_kwargs={'device': 'cuda:0'})

load INSTRUCTOR_Transformer
max_seq_length  512


### Creating DB

In [ ]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'

## Here is the nmew embeddings being used
embedding = instructor_embeddings

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

# persiste the db to disk
vectordb.persist()
vectordb = None

In [ ]:
# Now we can load the persisted database from disk, and use it as normal.
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding)

### Make a retriever

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

### Make a chain

In [ ]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [ ]:
## Cite sources

import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    print(wrap_text_preserve_newlines(llm_response['result']))
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [ ]:
%%time

# example
query = "How many players can battleline accomodate?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 The Battle Line game can accommodate two players.


Sources:
/content/drive/MyDrive/colab/AI Planet LLM Bootcamp/data/BattleLineRules.pdf
/content/drive/MyDrive/colab/AI Planet LLM Bootcamp/data/BattleLineRules.pdf
/content/drive/MyDrive/colab/AI Planet LLM Bootcamp/data/BattleLineRules.pdf


In [ ]:
%%time

# example
query = "How to win the game?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 To win the game, a player must either claim three adjacent Flags or any five Flags. The game ends immediately
when this happens, and the player who achieved the victory scores five points while the loser gets as many
points as the number of Flags he has claimed. If several games are played, the winner always scores five
points, and the loser gets as many points as the number of Flags he has claimed. The game can be played with
advanced rules, where players can use


Sources:
/content/drive/MyDrive/colab/AI Planet LLM Bootcamp/data/BattleLineRules.pdf
/content/drive/MyDrive/colab/AI Planet LLM Bootcamp/data/BattleLineRules.pdf
/content/drive/MyDrive/colab/AI Planet LLM Bootcamp/data/BattleLineRules.pdf
CPU times: user 7min 3s, sys: 1.08 s, total: 7min 4s
Wall time: 4min 46s


In [ ]:
%%time

# example
query = "Who designed battleline game"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Reiner Knizia


Sources:
/content/drive/MyDrive/colab/AI Planet LLM Bootcamp/data/BattleLineRules.pdf
/content/drive/MyDrive/colab/AI Planet LLM Bootcamp/data/BattleLineRules.pdf
/content/drive/MyDrive/colab/AI Planet LLM Bootcamp/data/BattleLineRules.pdf
CPU times: user 6min 3s, sys: 885 ms, total: 6min 4s
Wall time: 4min 3s


In [ ]:
%%time

# example
query = "Can you explain how to play Battleline?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Sure! Battleline is a card game where two players face off against each other in an ancient battle. Each
player starts with 60 Troop cards and 60 Tactics cards, which are divided into six different colors. The goal
of the game is to win by having more Flags on your side of the battle line at the end of the game.

Players take turns selecting either one Troop card or one Tactics card from their hand and placing it face


Sources:
/content/drive/MyDrive/colab/AI Planet LLM Bootcamp/data/BattleLineRules.pdf
/content/drive/MyDrive/colab/AI Planet LLM Bootcamp/data/BattleLineRules.pdf
/content/drive/MyDrive/colab/AI Planet LLM Bootcamp/data/BattleLineRules.pdf
CPU times: user 7min 10s, sys: 1.12 s, total: 7min 11s
Wall time: 4min 41s


In [ ]:
%%time

# example
query = "Can you explain how to play Battleline?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Battleline is a two-player strategy game where players aim to create powerful formations on their side of the
battle line to beat the formations on the opponent's side of the respective Flags. The first player to win
three adjacent Flags (a Breakthrough) or any five Flags (an Envelopment) achieves victory. Players take turns
selecting either one Troop card or one Tactics card from their hand and placing it face up on their side of
the battle line. At the end of their turn, they draw one card to refresh their hand to seven. The Flags are
won by formations of Troop cards played adjacent to the Flags. Tactics cards can be used to influence the
formations.


Sources:
/content/drive/MyDrive/colab/AI Planet LLM Bootcamp/data/BattleLineRules.pdf
/content/drive/MyDrive/colab/AI Planet LLM Bootcamp/data/BattleLineRules.pdf
/content/drive/MyDrive/colab/AI Planet LLM Bootcamp/data/BattleLineRules.pdf
CPU times: user 18min 52s, sys: 11min 23s, total: 30min 16s
Wall time: 1h 3min 59s
